In [ ]:
import pandas as pd

# Carregar os DataFrames
df2013 = pd.read_excel('./BaseLimpo/2013_coleta_tipos_residuos.xlsx')
df2014 = pd.read_excel('./BaseLimpo/2014_coleta_tipos_residuos.xlsx')
df2015 = pd.read_excel('./BaseLimpo/2015_coleta_tipos_residuos.xlsx')
df2016 = pd.read_excel('./BaseLimpo/2016_coleta_tipos_residuos.xlsx')
df2017 = pd.read_excel('./BaseLimpo/2017_coleta_tipos_residuos.xlsx')
df2018 = pd.read_excel('./BaseLimpo/2018_coleta_tipos_residuos.xlsx')
df2019 = pd.read_excel('./BaseLimpo/2019_coleta_tipos_residuos.xlsx')
df2020 = pd.read_excel('./BaseLimpo/2020_coleta_tipos_residuos.xlsx')
df2021 = pd.read_excel('./BaseLimpo/2021_coleta_tipos_residuos.xlsx')
df2023 = pd.read_excel('./BaseLimpo/2023_coleta_tipos_residuos.xlsx')
df2024 = pd.read_excel('./BaseLimpo/2024_coleta_tipos_residuos.xlsx')


# Criar uma lista com os DataFrames, excluindo o de 2022
dfs = [df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020, df2021]

# Inicializar o DataFrame final com o primeiro DataFrame (df2013)
df = df2013[['tipo_residuo', 'total_2013']]


for i,x in enumerate(dfs[1:], start=2014):
    coluna_nome = f'total_{i}'
            
    # Realizar o merge (junção) com base na coluna 'tipo_residuo
    df = pd.merge(df, x[['tipo_residuo', coluna_nome]], on='tipo_residuo', how='outer')
    

dfs2 = [df2023, df2024]

for i,y in enumerate(dfs2[0:], start=2023):
    coluna_nome = f'total_{i}'
            
    # Realizar o merge (junção) com base na coluna 'tipo_residuo
    df = pd.merge(df, y[['tipo_residuo', coluna_nome]], on='tipo_residuo', how='outer')





### Tratamento

In [ ]:
# Preenchendo os NaN dos números
for ano in range(2013, 2025):
    if ano != 2022:
        df[f'total_{ano}'] = df[f'total_{ano}'].fillna(0)



In [ ]:
# Deletando a ultima linha que é tudo NaN
df = df.iloc[:-1]

In [ ]:
# Padronizando a coluna tipo_residuo

df['tipo_residuo'] = df['tipo_residuo'].str.strip().str.lower()

In [ ]:
# Mudando os nomes 

df['tipo_residuo'] = df['tipo_residuo'].replace({
    'alim vencidos': 'alimentos_vencidos',
    'desconsiderando escoria, chorume, cancelamentos e saï¿½das de resï¿½duos': 'nao_classificados',
    'diversos-emae': 'diversos_emae',
    'entulho apreendido': 'entulho_apreendido',
    'entulho do predio': 'entulho_predial',
    'entulho manual': 'entulho_manual',
    'entulho mecanizado': 'entulho_mecanizado',
    'equipe de eventos especiais e operacoes de emergencia': 'eventos_e_emergencias',
    'feira livre': 'feira_livre',
    'gg saude': 'saude_geral',
    'limpeza urbana': 'limpeza_urbana',
    'material apreendido': 'material_apreendido',
    'patio compostagem': 'compostagem',
    'pg saude': 'saude_publica',
    'rejeito-cmt carolina maria de jesus': 'rejeitos_cmj', #repetido A
    'rejeito-cmt cmj': ' rejeitos_cmj', #repetido A
    'remocao residuos descartados': 'remocao_residuos', 
    'residuo de acumulador': 'acumuladores',
    'residuo de desfazimento': 'desfazimentos',
    'residuo ecoponto - entulho': 'ecoponto_entulho',
    'residuos de boca de lobo': 'bocas_de_lobo',
    'residuos de corregos': 'corregos',
    'residuos de ecoponto': 'ecoponto',
    'residuos de ecoponto - diversos': 'ecoponto_diversos',
    'residuos de piscinao': 'piscinoes',
    'residuos patio compostagem': 'patio_compostagem',
    'seletiva': 'coleta_seletiva',
    'varricao': 'varricao_manual',
    'varricao mecanizada ': 'varricao_mecanizada',
    'total geral' : 'total_geral'
    
})

df['tipo_residuo'] = df['tipo_residuo'].str.strip().str.lower()

In [ ]:
# Somando as duplicadas para não perder os dados

duplicadas = df[df.duplicated('tipo_residuo', keep=False)]
df.loc[duplicadas.index, df.columns != 'tipo_residuo'] = duplicadas.groupby('tipo_residuo', sort=False).transform('sum').loc[duplicadas.index]
df = df.drop_duplicates()

In [ ]:
# Deixando tudo com apenas 2 números depois do ponto
for ano in range(2013,2025):
    if ano != 2022:
        df[f'total_{ano}'] = df[f'total_{ano}'].round(2)

In [ ]:
# Adicionando a linha 'total_gera

df = pd.concat([df[df['tipo_residuo'] != 'total_geral'], df[df['tipo_residuo'] == 'total_geral']], ignore_index=True)

colunas_para_somar = ['total_2013', 'total_2014', 'total_2015', 'total_2016', 'total_2017', 'total_2018', 'total_2019', 'total_2020']

# Calcular a soma das colunas especificadas
soma_colunas = df[colunas_para_somar].sum()

# Atualizar os valores da linha "total_geral"
df.loc[df['tipo_residuo'] == 'total_geral', colunas_para_somar] = soma_colunas.values

In [ ]:
# Criando total_anos

df['total_anos'] = df.iloc[:, 1:].select_dtypes(include='number').sum(axis=1)

In [ ]:
# Salvar em excel

df.to_excel('tipo_residuos_total.xlsx', index=False)

x = df.drop(columns=['total_anos']).select_dtypes(include='number').iloc[-1]
df_soma = x.values
print(x)